<a href="https://colab.research.google.com/github/RomaricKanyamibwa/Normalisation-de-concepts-medicaux/blob/master/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.6.0)


In [0]:

import pandas as pd
#import collections
from pprint import pprint  # pretty-printer
import gensim
from gensim.models import word2vec
from gensim.parsing.preprocessing import remove_stopwords
import logging

In [0]:
%pwd
%cd /home/
%ls -l

/home
total 4
drwxr-xr-x 2 root root 4096 Dec  8 15:38 drive/


In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22298 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu2~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu2~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu2~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleus

In [0]:
!mkdir drive
!google-drive-ocamlfuse drive
!ls drive/
!cd drive/Coolab/
!ls -l

mkdir: cannot create directory ‘drive’: File exists
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
 code
'Colab Notebooks'
 Coolab
 deeplearningbook.pdf
 drive-download-20181103T154811Z-001.zip
 kusnerb15.pdf
 norm
 raw
 sujet-normalisation.odp
 tokenized
 Training_test
 Word2VecModels
 word2vec-sg-s0100-w08-m004-ns05-s0.001-a0.025-i05.zip
 word2vec-sg-s0200-w08-m004-ns05-s0.001-a0.025-i05.zip
total 4
drwxr-xr-x 2 root root 4096 Dec  8 15:38 drive


In [0]:
google_drive="drive/Coolab/"
df_disorders=pd.read_csv("drive/Coolab/Disorders.csv",names=["Code","Disorder","GenDisorder"],sep="\t") 
Codes=df_disorders['Code']#Y vector (outputs)
Disorders=df_disorders['Disorder']#X vector(test)
GenDisorders=df_disorders['GenDisorder']#X vector (train)

Disorders=[m+" "+n for m,n in zip(Disorders,GenDisorders)]
print(Disorders[30])
for i,j in enumerate(zip(Codes,Disorders)):
  if('postaural' in j):
    print(j)

postaural fistula postauricular fistula


In [0]:
def input_tokenize(input_list):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_list))
    
    for i, line in enumerate (input_list): 
        if (i%10000==0):
            logging.info ("read {0} reviews".format (i))
        # do some pre-processing and return a list of words for each review text
        line=remove_stopwords(line)
        yield gensim.utils.simple_preprocess(line,min_len=3)

In [0]:
logging.info ("Tokenization")
Disorders_corpus=(input_tokenize(Disorders))
GenDisorders_corpus=(input_tokenize(GenDisorders))
logging.info ("Done Tokenization")

Corpus=[[m]+n for m,n in zip(Codes,Disorders_corpus)]
#print(Disorders_corpus[30])

In [0]:
print("DisordrersLen:",len(Disorders))
print("CodesLen:",len(Codes))

DisordrersLen: 623509
CodesLen: 623509


In [0]:

#for i,k in enumerate(Corpus):
 # print(k)
  #if i>10:break

print("Iterator")
iter_Corpus=iter(Corpus)
for i,k in enumerate(iter_Corpus):
  if  i==30:
    print(k)

Iterator
['C0395905', 'postaural', 'fistula', 'postauricular', 'fistula']


In [0]:
def gensim_demo(sentences):
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    model = word2vec.Word2Vec(sentences, iter=20, min_count=1, size=300, workers=6)

    # get the word vector of "the"
    #print(model.wv['the'])

    # get the most common words
    print(model.wv.index2word[0], model.wv.index2word[1], model.wv.index2word[2])

    # get the least common words
    vocab_size = len(model.wv.vocab)
    print(model.wv.index2word[vocab_size - 1], model.wv.index2word[vocab_size - 2], model.wv.index2word[vocab_size - 3])

    # find the index of the 2nd most common word ("of")
#    print('Index of "of" is: {}'.format(model.wv.vocab['of'].index))

    # some similarity fun
    print(model.wv.similarity('fracture', 'bone'), model.wv.similarity('broken', 'arm'))

    # what doesn't fit?
    print(model.wv.doesnt_match("green blue red zebra".split()))

#    str_data = read_data(root_path + filename)
#    index_data = convert_data_to_index(str_data, model.wv)
#    print(str_data[:4], index_data[:4])

    # save and reload the model
    model.save("drive/Coolab/Disorders")
    return model



In [0]:

#print("size corpus:",len(list(iter_Corpus)))
logging.info ("Tokenization")
Disorders_corpus=(input_tokenize(Disorders))
GenDisorders_corpus=(input_tokenize(GenDisorders))
logging.info ("Done Tokenization")
#pense à cree des fichier avec les donées

class MyCorpus(object):
  def __init__(self,Codes,GenDisorders_corpus):
    self.Codes = Codes
    self.GenDisorders_corpus=GenDisorders_corpus

  def __iter__(self):
    for m,n in zip(self.Codes,self.GenDisorders_corpus):
      yield [m]+n

#Corpus=MyCorpus(Codes,GenDisorders_corpus)
Corpus=[[m]+n for m,n in zip(Codes,Disorders_corpus)]
print("Corpus")
for i,k in enumerate(Corpus):
  print(k)
  if i>5:break

iter_Corpus=(Corpus)

model=gensim_demo(Corpus)


Corpus
['C0423346', 'cupuliform', 'cataract', 'cupuliform', 'cataract']
['C4510595', 'acquired', 'hallux', 'limitus', 'right', 'great', 'toe', 'acquired', 'hallux', 'limitus', 'right', 'great', 'toe']
['C1394389', 'cyst', 'ora', 'serrata', 'cyst', 'ora', 'serrata', 'retina']
['C1394389', 'cyst', 'ora', 'serrata', 'cyst', 'ora', 'serrata', 'retina']
['C1394389', 'cyst', 'ora', 'serrata', 'cyst', 'ora', 'serrata', 'retina']
['C1394389', 'ora', 'serrata', 'cyst', 'cyst', 'ora', 'serrata', 'retina']
['C1394389', 'ora', 'serrata', 'cyst', 'cyst', 'ora', 'serrata', 'retina']
fracture unspecified encounter
C2214472 C0857024 C2081567
0.60549426 -0.040414672


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


zebra


In [0]:
model.most_similar('fracture')

#model.most_similar('C3714535')


In [0]:
len(model.wv['postaural'])
#(model.wv.vocab)

300